### classify stackoverflow posts by considering each document a simple mean of its individual word embeddings

**TODO**
 - use weighted averages (tf-id, ppmi, etc)
 - use a different scoring function so that wrong guesses that are close to the correct values (think "python", "python-2") are penalized less than guesses that are wildly wrong.


In [1]:
import csv
import os
import re
import sys
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Model, Sequential

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

# my stuff in the helpers/ directory
from helpers import files,texts, metrics

Using TensorFlow backend.


In [2]:
np.random.seed(seed=42)

In [3]:
texts, labels = files.read_stackoverflow_sample_small()

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100           

In [5]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [6]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(labels)

In [7]:
# word => embedding
embeddings_index = files.read_glove_100()

In [8]:
# word_position => embedding for that word
numeric_embeddings_index = {}

embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue  
    
    embedding_vector = embeddings_index.get(word)
    
    numeric_embeddings_index[i] = embedding_vector
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [9]:
inverse_word_index = {}

for word,i in word_index.items():
    inverse_word_index[i] = word

In [10]:
document_embeddings = []

for i,document_sequence in enumerate(sequences):
    
    individual_word_embeddings = []
    
    for word_position in document_sequence:       
                                  
        word = inverse_word_index[word_position]
        
        word_embedding = embeddings_index.get(word)    

        if word_embedding is None:
            continue
        else:    
            individual_word_embeddings.append(word_embedding)
         
    if len(individual_word_embeddings) == 0:
        # some documents have no text
        # so we assign a zero vector
        individual_word_embeddings = np.zeros(
            EMBEDDING_DIM,dtype='float32').reshape(1,EMBEDDING_DIM)
        
    document_embedding = np.mean(individual_word_embeddings,axis=0)
    
    document_embeddings.append(document_embedding)

In [11]:
data = np.vstack(document_embeddings)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((18822, 100), (18822, 5599))

In [12]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()
model.add(Dense(512,input_dim=EMBEDDING_DIM))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')
# multi label needs a specific accuracy function

model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_val,y_val))

Train on 15058 samples, validate on 3764 samples
Epoch 1/10
15058/15058 [==============================] - 6s - loss: 0.0114 - val_loss: 0.0035
Epoch 2/10
15058/15058 [==============================] - 6s - loss: 0.0035 - val_loss: 0.0035
Epoch 3/10
15058/15058 [==============================] - 6s - loss: 0.0035 - val_loss: 0.0035
Epoch 4/10
15058/15058 [==============================] - 6s - loss: 0.0035 - val_loss: 0.0035
Epoch 5/10
15058/15058 [==============================] - 6s - loss: 0.0035 - val_loss: 0.0035
Epoch 6/10
15058/15058 [==============================] - 6s - loss: 0.0035 - val_loss: 0.0035
Epoch 7/10
15058/15058 [==============================] - 6s - loss: 0.0035 - val_loss: 0.0034
Epoch 8/10
15058/15058 [==============================] - 6s - loss: 0.0034 - val_loss: 0.0033
Epoch 9/10
15058/15058 [==============================] - 6s - loss: 0.0033 - val_loss: 0.0033
Epoch 10/10
15058/15058 [==============================] - 6s - loss: 0.0033 - val_loss: 0.0032


In [13]:
actual_label_indices = y_val[0].reshape(1,-1)

predicted_label_indices = np.zeros(labels_1.shape[1]).reshape(1,-1)
bool_vec = model.predict(X_val[0].reshape(1,-1)) > 0.1

predicted_label_indices[bool_vec] = 1

In [14]:
actual_labels = lb.inverse_transform(actual_label_indices)
predicted_labels = lb.inverse_transform(predicted_label_indices)

actual_labels,predicted_labels

([('keyboard-shortcuts', 'visual-studio', 'visual-studio-2008')],
 [('asp.net', 'javascript')])

In [15]:
X_val.shape

(3764, 100)

In [16]:
X_test_list = []

for validation_features in X_val:
    X_test_list.append(validation_features.reshape(1,-1))

X_test = np.vstack(X_test_list)    

In [18]:
micro_f1 = metrics.calculate_multilabel_metrics(model,X_val, y_val, threshold=0.08)
print('micro-averaged F1 score (validation set) is {0}'.format(micro_f1))

micro_prec = metrics.calculate_multilabel_metrics(model,X_val, y_val, metric='precision',threshold=0.08)
print('micro-averaged precision score (validation set) is {0}'.format(micro_prec))

micro_rec = metrics.calculate_multilabel_metrics(model,X_val, y_val, metric='recall',threshold=0.08)
print('micro-averaged recall score (validation set) is {0}'.format(micro_rec))


micro-averaged F1 score (validation set) is 0.15631046119235092
micro-averaged precision score (validation set) is 0.15396206346392483
micro-averaged recall score (validation set) is 0.15873160924792104
